# SciTeX Framework Server - Code Templates and Structure

This notebook demonstrates the Framework Server's capabilities for converting scripts to SciTeX framework patterns, managing boilerplate code, and ensuring proper project structure.

## 1. Overview of Framework Server

The Framework Server helps transform standard Python scripts into properly structured SciTeX projects with:
- Standard headers and imports
- Main function patterns
- Argument parsing
- Error handling
- Reproducibility features

In [ ]:
# Example of a typical research script before SciTeX
basic_script = """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Load and process data
data = pd.read_csv('experiment_data.csv')
filtered = signal.savgol_filter(data['signal'], 51, 3)

# Calculate metrics
mean_val = np.mean(filtered)
std_val = np.std(filtered)
print(f"Mean: {mean_val}, Std: {std_val}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(data['time'], data['signal'], alpha=0.5, label='Raw')
plt.plot(data['time'], filtered, label='Filtered')
plt.xlabel('Time (s)')
plt.ylabel('Signal')
plt.legend()
plt.savefig('results.png')
plt.show()
"""

print("Original script:")
print(basic_script)

## 2. Basic Framework Conversion

The Framework Server converts this to a properly structured SciTeX script:

In [ ]:
# Framework Server output
scitex_script = '''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# Timestamp: "2025-07-02 10:00:00 (username)"
# File: ./analyze_experiment.py
# ========================================
"""Analyze experimental data with filtering and visualization.

This script processes experimental time series data by:
1. Loading data from CSV
2. Applying Savitzky-Golay filtering
3. Computing statistics
4. Generating visualizations
"""
# ========================================
import os
import sys
from scitex import logging
from typing import Dict, Any

__FILE__ = "./analyze_experiment.py"
__DIR__ = os.path.dirname(__FILE__)
sys.path.append(os.path.dirname(__DIR__))
# ========================================

import scitex as stx
from scipy import signal

# ========================================
# CONFIG
# ========================================
CONFIG = stx.io.load_config(f"{__DIR__}/config/PATH.yaml")
PARAMS = stx.io.load_config(f"{__DIR__}/config/PARAMS.yaml")

# ========================================
# FUNCTIONS
# ========================================
def process_signal(data: pd.DataFrame, params: Dict[str, Any]) -> np.ndarray:
    """Apply Savitzky-Golay filter to signal data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Input data with 'signal' column
    params : dict
        Filter parameters
        
    Returns
    -------
    np.ndarray
        Filtered signal
    """
    return signal.savgol_filter(
        data['signal'], 
        params['window_length'], 
        params['polyorder']
    )

def compute_statistics(signal: np.ndarray) -> Dict[str, float]:
    """Compute signal statistics."""
    return {
        'mean': np.mean(signal),
        'std': np.std(signal),
        'min': np.min(signal),
        'max': np.max(signal)
    }

# ========================================
# MAIN
# ========================================
def main(args: argparse.Namespace) -> int:
    """Main analysis pipeline.
    
    Parameters
    ----------
    args : argparse.Namespace
        Command line arguments
        
    Returns
    -------
    int
        Exit code (0 for success)
    """
    # Setup logging
    stx.utils.setup_logging(args.log_level)
    logger = logging.getLogger(__name__)
    
    try:
        # Load data
        logger.info(f"Loading data from {CONFIG['data']['input']}")
        data = stx.io.load(CONFIG['data']['input'])
        
        # Process signal
        logger.info("Applying filter...")
        filtered = process_signal(data, PARAMS['filter'])
        
        # Compute statistics
        stats = compute_statistics(filtered)
        logger.info(f"Statistics: {stats}")
        
        # Visualization
        logger.info("Creating visualization...")
        fig, ax = stx.plt.subplots(figsize=PARAMS['plot']['figure_size'])
        
        # Plot raw and filtered signals
        ax.plot(data['time'], data['signal'], 
                alpha=0.5, label='Raw', color=COLORS['raw'])
        ax.plot(data['time'], filtered, 
                label='Filtered', color=COLORS['filtered'])
        
        ax.set_xyt('Time (s)', 'Signal', 
                   f'Experimental Data (mean={stats["mean"]:.2f})')
        ax.legend()
        
        # Save outputs
        output_path = CONFIG['output']['figures'] / f"results_{args.suffix}.png"
        stx.io.save(fig, output_path, dpi=PARAMS['plot']['dpi'])
        
        # Save statistics
        stats_path = CONFIG['output']['results'] / f"stats_{args.suffix}.yaml"
        stx.io.save(stats, stats_path)
        
        logger.info("Analysis complete!")
        return 0
        
    except Exception as e:
        logger.error(f"Analysis failed: {e}")
        return 1

# ========================================
# ARGUMENT PARSER
# ========================================
def get_parser() -> argparse.ArgumentParser:
    """Create command line argument parser."""
    parser = argparse.ArgumentParser(
        description=__doc__,
        formatter_class=argparse.RawDescriptionHelpFormatter
    )
    
    parser.add_argument(
        '--suffix', 
        type=str, 
        default='',
        help='Suffix for output files'
    )
    
    parser.add_argument(
        '--log-level',
        choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
        default='INFO',
        help='Logging level'
    )
    
    return parser

# ========================================
# ENTRY POINT
# ========================================
if __name__ == "__main__":
    import argparse
    parser = get_parser()
    args = parser.parse_args()
    sys.exit(main(args))

# EOF
'''

print("SciTeX framework script:")
print(scitex_script[:2000] + "\n... (truncated for display)")

## 3. Framework Components

The Framework Server manages several key components:

In [ ]:
# 1. Standard Header
header_template = '''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# Timestamp: "{timestamp} ({username})"
# File: {filepath}
# ========================================
"""{docstring}"""
# ========================================
'''

print("Standard header template:")
print(header_template)

In [ ]:
# 2. Import Organization
import_structure = """
# Standard library imports
import os
import sys
from scitex import logging
from typing import Dict, List, Any, Optional

# Path setup
__FILE__ = "{filepath}"
__DIR__ = os.path.dirname(__FILE__)
sys.path.append(os.path.dirname(__DIR__))

# Third-party imports
import numpy as np
import pandas as pd

# SciTeX imports
import scitex as stx

# Local imports
from .utils import helper_function
"""

print("Import organization pattern:")
print(import_structure)

In [ ]:
# 3. Configuration Loading Pattern
config_pattern = """
# ========================================
# CONFIG
# ========================================
CONFIG = stx.io.load_config(f"{__DIR__}/config/PATH.yaml")
PARAMS = stx.io.load_config(f"{__DIR__}/config/PARAMS.yaml")
COLORS = stx.io.load_config(f"{__DIR__}/config/COLORS.yaml")

# Optional: Environment-specific overrides
env = os.getenv('SCITEX_ENV', 'development')
if os.path.exists(f"{__DIR__}/config/PARAMS.{env}.yaml"):
    env_params = stx.io.load_config(f"{__DIR__}/config/PARAMS.{env}.yaml")
    PARAMS = stx.utils.merge_dicts(PARAMS, env_params)
"""

print("Configuration loading pattern:")
print(config_pattern)

## 4. Function Templates

The Framework Server provides templates for common function patterns:

In [ ]:
# Data processing function template
function_templates = {
    "data_processor": '''
def process_data(
    data: pd.DataFrame,
    params: Dict[str, Any],
    verbose: bool = False
) -> pd.DataFrame:
    """Process experimental data according to parameters.
    
    Parameters
    ----------
    data : pd.DataFrame
        Input data with required columns
    params : dict
        Processing parameters
    verbose : bool, optional
        Print progress information
        
    Returns
    -------
    pd.DataFrame
        Processed data
        
    Raises
    ------
    ValueError
        If required columns are missing
    """
    # Validate inputs
    required_cols = params.get('required_columns', [])
    missing = set(required_cols) - set(data.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    
    # Process data
    result = data.copy()
    
    # Your processing logic here
    
    return result
''',
    "analysis_function": '''
@stx.decorators.timed
@stx.decorators.log_function
def analyze_results(
    results: Dict[str, np.ndarray],
    method: str = 'standard'
) -> Dict[str, Any]:
    """Analyze experimental results.
    
    This function is decorated to:
    - Log execution time
    - Log function calls
    """
    analysis = {}
    
    # Analysis logic
    
    return analysis
''',
    "visualization_function": '''
def create_figure(
    data: pd.DataFrame,
    style: str = 'publication'
) -> Tuple[plt.Figure, plt.Axes]:
    """Create publication-ready figure."""
    # Set style
    stx.plt.set_style(style)
    
    # Create figure
    fig, axes = stx.plt.subplots(
        nrows=2, 
        ncols=2,
        figsize=PARAMS['figure']['size'],
        constrained_layout=True
    )
    
    # Plotting logic
    
    return fig, axes
''',
}

print("Function templates:")
for name, template in function_templates.items():
    print(f"\n=== {name} ===")
    print(template)

## 5. Main Function Patterns

Different main function patterns for different types of scripts:

In [ ]:
# Analysis script main function
analysis_main = '''
def main(args: argparse.Namespace) -> int:
    """Main analysis pipeline."""
    # Setup
    stx.utils.setup_logging(args.log_level)
    logger = logging.getLogger(__name__)
    
    # Create output directory
    output_dir = stx.path.Path(CONFIG['output']['base']) / args.experiment_id
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save command for reproducibility
    stx.repro.save_command(output_dir / 'command.txt')
    
    try:
        # Load data
        logger.info("Loading data...")
        data = stx.io.load(args.input_file)
        
        # Process
        logger.info("Processing...")
        results = process_data(data, PARAMS)
        
        # Save results
        logger.info("Saving results...")
        stx.io.save(results, output_dir / 'results.pkl')
        
        # Generate report
        if args.generate_report:
            generate_report(results, output_dir)
            
        logger.info("Analysis complete!")
        return 0
        
    except Exception as e:
        logger.error(f"Analysis failed: {e}", exc_info=True)
        return 1
'''

print("Analysis script main function:")
print(analysis_main)

In [ ]:
# Experiment script with multiple runs
experiment_main = '''
def run_single_experiment(
    params: Dict[str, Any],
    run_id: int
) -> Dict[str, Any]:
    """Run a single experiment iteration."""
    # Set random seed for reproducibility
    stx.utils.set_random_seed(params['seed'] + run_id)
    
    # Run experiment
    results = {
        'run_id': run_id,
        'params': params,
        'timestamp': stx.dt.now(),
    }
    
    # Your experiment logic here
    
    return results

def main(args: argparse.Namespace) -> int:
    """Run multiple experiments with parameter sweeps."""
    # Setup
    stx.utils.setup_logging(args.log_level)
    logger = logging.getLogger(__name__)
    
    # Load parameter grid
    param_grid = stx.io.load(args.param_file)
    
    # Run experiments
    results = []
    with stx.utils.tqdm(total=len(param_grid) * args.n_runs) as pbar:
        for params in param_grid:
            for run_id in range(args.n_runs):
                logger.info(f"Running experiment {run_id} with {params}")
                
                result = run_single_experiment(params, run_id)
                results.append(result)
                
                # Save intermediate results
                if args.save_intermediate:
                    stx.io.save(
                        results, 
                        CONFIG['output']['intermediate'] / 'results.pkl'
                    )
                
                pbar.update(1)
    
    # Aggregate results
    summary = aggregate_results(results)
    stx.io.save(summary, CONFIG['output']['final'] / 'summary.pkl')
    
    return 0
'''

print("Experiment script with multiple runs:")
print(experiment_main)

## 6. Error Handling and Logging

The Framework Server ensures proper error handling and logging:

In [ ]:
# Comprehensive error handling
error_handling_pattern = '''
import scitex as stx
from scitex import logging
from typing import Optional

class AnalysisError(Exception):
    """Custom exception for analysis errors."""
    pass

def safe_analysis(
    data: pd.DataFrame,
    fallback: Optional[str] = None
) -> Dict[str, Any]:
    """Perform analysis with comprehensive error handling."""
    logger = logging.getLogger(__name__)
    
    try:
        # Validate inputs
        if data.empty:
            raise AnalysisError("Input data is empty")
            
        # Main analysis
        logger.debug(f"Analyzing {len(data)} samples")
        results = perform_analysis(data)
        
        # Validate results
        if not validate_results(results):
            raise AnalysisError("Results validation failed")
            
        return results
        
    except AnalysisError as e:
        logger.error(f"Analysis error: {e}")
        if fallback:
            logger.info(f"Using fallback method: {fallback}")
            return perform_fallback_analysis(data, fallback)
        raise
        
    except Exception as e:
        logger.error(f"Unexpected error: {e}", exc_info=True)
        
        # Save debug information
        debug_dir = stx.path.Path("./debug")
        debug_dir.mkdir(exist_ok=True)
        
        stx.io.save({
            'error': str(e),
            'data_shape': data.shape,
            'data_sample': data.head(),
            'traceback': stx.utils.get_traceback()
        }, debug_dir / f"error_{stx.dt.now().strftime('%Y%m%d_%H%M%S')}.pkl")
        
        raise

def setup_logging_with_file(args: argparse.Namespace):
    """Setup logging to both console and file."""
    log_dir = stx.path.Path(CONFIG['output']['logs'])
    log_dir.mkdir(exist_ok=True)
    
    log_file = log_dir / f"{args.experiment_id}_{stx.dt.now().strftime('%Y%m%d_%H%M%S')}.log"
    
    logging.basicConfig(
        level=getattr(logging, args.log_level),
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
'''

print("Error handling and logging patterns:")
print(error_handling_pattern)

## 7. Testing Framework Integration

The Framework Server can generate test templates:

In [ ]:
# Test template generation
test_template = '''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# Test file for analyze_experiment.py

import pytest
import numpy as np
import pandas as pd
import scitex as stx
from pathlib import Path

# Import the module to test
import sys
sys.path.append('..')
from analyze_experiment import process_signal, compute_statistics

class TestSignalProcessing:
    """Test signal processing functions."""
    
    @pytest.fixture
    def sample_data(self):
        """Create sample data for testing."""
        time = np.linspace(0, 10, 1000)
        signal = np.sin(2 * np.pi * time) + 0.1 * np.random.randn(1000)
        return pd.DataFrame({'time': time, 'signal': signal})
    
    @pytest.fixture
    def test_params(self):
        """Test parameters."""
        return {
            'window_length': 51,
            'polyorder': 3
        }
    
    def test_process_signal(self, sample_data, test_params):
        """Test signal processing."""
        filtered = process_signal(sample_data, test_params)
        
        # Check output shape
        assert len(filtered) == len(sample_data)
        
        # Check smoothing effect
        assert np.std(filtered) < np.std(sample_data['signal'])
    
    def test_compute_statistics(self, sample_data):
        """Test statistics computation."""
        stats = compute_statistics(sample_data['signal'].values)
        
        # Check required keys
        assert all(key in stats for key in ['mean', 'std', 'min', 'max'])
        
        # Check value ranges
        assert stats['min'] <= stats['mean'] <= stats['max']
        assert stats['std'] >= 0
    
    @pytest.mark.parametrize("window_length,polyorder", [
        (11, 3),
        (51, 3),
        (101, 5),
    ])
    def test_filter_parameters(self, sample_data, window_length, polyorder):
        """Test different filter parameters."""
        params = {
            'window_length': window_length,
            'polyorder': polyorder
        }
        
        filtered = process_signal(sample_data, params)
        assert filtered is not None
        assert not np.any(np.isnan(filtered))

class TestIntegration:
    """Integration tests."""
    
    def test_full_pipeline(self, tmp_path):
        """Test the full analysis pipeline."""
        # Create temporary config
        config_dir = tmp_path / "config"
        config_dir.mkdir()
        
        # Run main with test arguments
        # ...

if __name__ == "__main__":
    pytest.main([__file__, "-v"])
'''

print("Test template:")
print(test_template)

## 8. Project Scaffolding

The Framework Server can generate complete project structures:

In [ ]:
# Project structure generation
project_structure = """
my_research_project/
├── README.md
├── pyproject.toml
├── .gitignore
├── config/
│   ├── PATH.yaml
│   ├── PARAMS.yaml
│   ├── COLORS.yaml
│   └── PARAMS.hpc.yaml  # HPC-specific overrides
├── src/
│   ├── __init__.py
│   ├── main.py          # Main analysis script
│   ├── preprocessing.py
│   ├── analysis.py
│   ├── visualization.py
│   └── utils.py
├── tests/
│   ├── __init__.py
│   ├── test_preprocessing.py
│   ├── test_analysis.py
│   └── conftest.py
├── notebooks/
│   ├── 01_exploration.ipynb
│   ├── 02_analysis.ipynb
│   └── 03_figures.ipynb
├── data/
│   ├── raw/            # Original data (gitignored)
│   ├── processed/      # Processed data (gitignored)
│   └── README.md
├── results/
│   ├── figures/
│   ├── tables/
│   └── logs/
└── scripts/
    ├── run_analysis.sh
    ├── run_on_hpc.sh
    └── setup_env.sh
"""

print("Generated project structure:")
print(project_structure)

# Example generated files
generated_files = {
    "README.md": """# My Research Project

## Overview
Brief description of your research project.

## Installation
```bash
pip install -e .
```

## Usage
```bash
python src/main.py --experiment-id exp001
```

## Project Structure
- `config/`: Configuration files
- `src/`: Source code
- `tests/`: Unit tests
- `notebooks/`: Jupyter notebooks
- `data/`: Data files (not tracked)
- `results/`: Output files
""",
    ".gitignore": """# Data files
data/raw/*
data/processed/*
!data/raw/README.md
!data/processed/README.md

# Results
results/*
!results/.gitkeep

# Python
__pycache__/
*.pyc
.pytest_cache/
*.egg-info/

# Jupyter
.ipynb_checkpoints/

# Environment
.env
venv/
""",
    "pyproject.toml": """[project]
name = "my-research-project"
version = "0.1.0"
description = "Research project using SciTeX framework"
dependencies = [
    "scitex>=2.0.0",
    "numpy",
    "pandas",
    "scipy",
    "matplotlib",
]

[project.optional-dependencies]
dev = [
    "pytest",
    "pytest-cov",
    "black",
    "flake8",
]

[tool.black]
line-length = 88
target-version = ["py38"]
""",
}

print("\nExample generated files:")
for filename, content in generated_files.items():
    print(f"\n=== {filename} ===")
    print(content[:300] + "..." if len(content) > 300 else content)

## 9. CLI Integration

The Framework Server helps create proper command-line interfaces:

In [ ]:
# Advanced CLI patterns
cli_pattern = '''
import argparse
import scitex as stx
from typing import List

def create_parser() -> argparse.ArgumentParser:
    """Create comprehensive CLI parser."""
    parser = argparse.ArgumentParser(
        prog='analyze',
        description='Analyze experimental data with SciTeX',
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog=\'\'\'Examples:
  %(prog)s data.csv                    # Basic analysis
  %(prog)s data.csv -o results/        # Specify output directory  
  %(prog)s data.csv --method advanced  # Use advanced method
  %(prog)s data.csv --config custom.yaml  # Custom config
  \'\'\''
    )
    
    # Positional arguments
    parser.add_argument(
        'input_file',
        type=str,
        help='Input data file (CSV or HDF5)'
    )
    
    # Optional arguments
    parser.add_argument(
        '-o', '--output-dir',
        type=str,
        default='./results',
        help='Output directory (default: %(default)s)'
    )
    
    parser.add_argument(
        '--method',
        choices=['basic', 'standard', 'advanced'],
        default='standard',
        help='Analysis method (default: %(default)s)'
    )
    
    parser.add_argument(
        '--config',
        type=str,
        help='Custom configuration file'
    )
    
    # Feature flags
    parser.add_argument(
        '--plot',
        action='store_true',
        help='Generate plots'
    )
    
    parser.add_argument(
        '--report',
        action='store_true',
        help='Generate PDF report'
    )
    
    # Advanced options
    advanced = parser.add_argument_group('advanced options')
    
    advanced.add_argument(
        '--parallel',
        action='store_true',
        help='Use parallel processing'
    )
    
    advanced.add_argument(
        '--n-jobs',
        type=int,
        default=-1,
        help='Number of parallel jobs (-1 for all CPUs)'
    )
    
    advanced.add_argument(
        '--seed',
        type=int,
        default=42,
        help='Random seed for reproducibility'
    )
    
    # Debugging
    debug = parser.add_argument_group('debugging')
    
    debug.add_argument(
        '--debug',
        action='store_true',
        help='Enable debug mode'
    )
    
    debug.add_argument(
        '--profile',
        action='store_true',
        help='Profile execution time'
    )
    
    debug.add_argument(
        '--log-level',
        choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'],
        default='INFO',
        help='Logging level'
    )
    
    return parser

def validate_args(args: argparse.Namespace) -> None:
    """Validate command line arguments."""
    # Check input file exists
    if not stx.path.Path(args.input_file).exists():
        raise FileNotFoundError(f"Input file not found: {args.input_file}")
    
    # Validate custom config
    if args.config and not stx.path.Path(args.config).exists():
        raise FileNotFoundError(f"Config file not found: {args.config}")
    
    # Check output directory
    output_path = stx.path.Path(args.output_dir)
    if output_path.exists() and not args.force:
        response = input(f"Output directory {output_path} exists. Overwrite? [y/N] ")
        if response.lower() != 'y':
            raise ValueError("Output directory exists. Use --force to overwrite.")
'''

print("Advanced CLI patterns:")
print(cli_pattern)

## Summary

The SciTeX Framework Server provides comprehensive support for:

1. **Script Conversion**: Transform basic scripts into well-structured SciTeX projects
2. **Boilerplate Management**: Standard headers, imports, and configuration loading
3. **Function Templates**: Common patterns for data processing, analysis, and visualization
4. **Error Handling**: Comprehensive error handling and logging patterns
5. **Testing Integration**: Generate test templates and fixtures
6. **Project Scaffolding**: Create complete project structures
7. **CLI Development**: Advanced command-line interface patterns
8. **Reproducibility**: Built-in support for reproducible research

This ensures that all SciTeX projects follow consistent patterns and best practices for scientific computing.